In [1]:
#Import required packages
import pandas as pd
import numpy as np
import pickle
import re
import itertools as it


import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#Matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 

#Seaborn
import seaborn as sns
sns.set()

#Sklearn
from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.metrics import median_absolute_error
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (accuracy_score,
                             f1_score,
                             roc_auc_score,
                             roc_curve,
                             confusion_matrix)

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import (RandomForestClassifier,
                              RandomForestClassifier,
                              RandomForestRegressor,
                              GradientBoostingClassifier, 
                              ExtraTreesClassifier, #For each feature split rule is random, not optimal
                              VotingClassifier, 
                              AdaBoostClassifier, 
                              BaggingRegressor)

from sklearn.model_selection import (cross_val_score,
                                     GridSearchCV,
                                     RandomizedSearchCV,
                                     learning_curve,
                                     validation_curve,
                                     train_test_split,
                                     cross_validate)
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

#XGBoost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

import os
import pickle


C:\Users\moham\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Open Corpus of News Article Text
# with open('./data/news_df.pickle', 'rb') as file:
# with open('./data/news_data_frame_reduced_preprocessed.pickle', 'rb') as file:
with open('./data/news_fake_real_df_reduced_token.pickle', 'rb') as file:
    news_data_frame = pickle.load(file)

In [3]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(news_data_frame.text)

### Prepare text grams

In [4]:
# Convert text and response to array 
x_text = news_data_frame.text.values
y_response = news_data_frame.Not_Real_or_Real.values

In [5]:
# function to recombine nested list of tokens into full articles 
def lemma_combine(lis):
    parsed_articles = []
    
    for i in range(len(lis)):
        concat_art = ' '.join(lis[i])
        parsed_articles.append(concat_art)
    
    return parsed_articles


In [6]:
# function to match "cleaned" text back up with response variable
def zip_response(observations, response):
    response = response.tolist()
    
    return list(zip(observations, response))

In [7]:
lem = news_data_frame['tokenized_text']

In [8]:
# Create bi-grams for our text 
phrases = Phrases(lem)
bigram = Phraser(phrases)
bigram_lem = list(bigram[lem])

2019-05-24 14:08:40,936 : INFO : collecting all words and their counts
2019-05-24 14:08:40,938 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-24 14:08:44,825 : INFO : collected 1021646 word types from a corpus of 1797618 words (unigram + bigrams) and 4426 sentences
2019-05-24 14:08:44,825 : INFO : using 1021646 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2019-05-24 14:08:44,827 : INFO : source_vocab length 1021646
2019-05-24 14:08:57,928 : INFO : Phraser built with 16848 16848 phrasegrams


In [9]:
# Create tri-grams for our text
phrases2 = Phrases(bigram_lem)
trigram = Phraser(phrases2)
trigram_lem = list(trigram[bigram_lem])

2019-05-24 14:09:03,463 : INFO : collecting all words and their counts
2019-05-24 14:09:03,465 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-24 14:09:06,635 : INFO : collected 1067409 word types from a corpus of 1589332 words (unigram + bigrams) and 4426 sentences
2019-05-24 14:09:06,636 : INFO : using 1067409 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2019-05-24 14:09:06,636 : INFO : source_vocab length 1067409
2019-05-24 14:09:21,594 : INFO : Phraser built with 22256 22256 phrasegrams


In [10]:
# Recombine full article text for unigrams, bigrams, and trigrams
uni_lem_comb = lemma_combine(lem)
bi_lem_comb = lemma_combine(bigram_lem)
tri_lem_comb = lemma_combine(trigram_lem)

In [11]:
token_uni = "./data/grams/token_unigram_text"
fileObject = open(token_uni,'wb') 
pickle.dump(lem,fileObject)   
fileObject.close()

In [12]:
token_trigrams = "./data/grams/token_trigram_text"
fileObject = open(token_trigrams,'wb') 
pickle.dump(trigram_lem,fileObject)   
fileObject.close()

In [13]:
# Save processed text so it can be loaded later 
unigrams = "./data/grams/unigram_text"
fileObject = open(unigrams,'wb') 
pickle.dump(uni_lem_comb,fileObject)   
fileObject.close()

In [14]:
bigrams = "./data/grams/bigram_text"
fileObject = open(bigrams,'wb') 
pickle.dump(bi_lem_comb,fileObject)   
fileObject.close()

In [15]:
trigrams = "./data/grams/trigram_text"
# open the file for writing
fileObject = open(trigrams,'wb')
pickle.dump(tri_lem_comb,fileObject)   
fileObject.close()